In [ ]:
!pwd

In [ ]:
%conda install python-dotenv wandb nbstripout -y

# Setting lerobot env
from lerobot_util import setup_lerobot_env

setup_lerobot_env()


In [ ]:
import json
import sys


from lerobot_util import cd_lerobot, get_lerobot_camera_index, print_shell_md

cd_lerobot()

# Training ACT model.
# Adjust the following variables before starting training
policy_type = 'diffusion'
hf_namespace = 'fanaustinca'
model_repo = f"{hf_namespace}/fll_diffusion_model_400episodes_v2"
eval_repo = f"{hf_namespace}/fll_eval_act_400episodes_v2"
robot_port = '/dev/ttyACM1'
num_episodes = 10

top_camera_index, wrist_camera_index = get_lerobot_camera_index()

# Zoom in 130 to focus on board
!v4l2-ctl -d /dev/video{top_camera_index} --set-ctrl=zoom_absolute=130

camera_config = {
  "wrist": {
    "type": "opencv", 
    "index_or_path": wrist_camera_index, 
    "fps": 30, 
    "width": 640, 
    "height": 480
  },
  "top": {
    "type": "opencv", 
    "index_or_path": top_camera_index, 
    "fps": 30, 
    "width": 640, 
    "height": 480
  }
}
camera_string = json.dumps(camera_config, separators=(',', ':'))

arguments = [
  "--robot.type=so101_follower",
  f"--robot.port={robot_port}",
  f"--robot.cameras='{camera_string}",
  f"--job_name=fll_eval_{policy_type}",
  "--policy.device=cuda",
  "--policy.use_amp=true",
  "--policy.push_to_hub=true",
  f"--policy.type={policy_type}",
  f"--policy.path={model_repo}",
  f"--dataset.repo_id={eval_repo}",
  f"--dataset.num_episodes={num_episodes}",
  "--dataset.single_task=\"fll_mission\"",
  "--display_data=true"
  ]

if policy_type == 'diffusion':
    arguments += ["--policy.num_inference_steps=30"]

print_shell_md(
  "Run following command to evaluate model.", 
  f"{sys.executable}",
  *arguments
)
